In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

FILES_PATH = "./files" if os.environ.get("HW_ENV") == "mac" else "/kaggle"

print("hey")
for dirname, _, filenames in os.walk(f"{FILES_PATH}/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train_set = pd.read_csv(f"{FILES_PATH}/input/train.csv")
test_set = pd.read_csv(f"{FILES_PATH}/input/test.csv")
test_set.describe()

hey
./files/input/test.csv
./files/input/train.csv
./files/input/sample_submission.csv


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,28000.0,...,28000.000000,28000.000000,28000.000000,28000.000000,28000.000000,28000.0,28000.0,28000.0,28000.0,28000.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.164607,0.073214,0.028036,0.011250,0.006536,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.473293,3.616811,1.813602,1.205211,0.807475,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,253.000000,254.000000,193.000000,187.000000,119.000000,0.0,0.0,0.0,0.0,0.0


In [4]:
train_x, train_y = train_set.iloc[:, 1:], train_set.iloc[:, :1]

train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
test_x = test_set.to_numpy()

train_x = train_x.astype("float32") / 255.0
test_x = test_x.astype("float32") / 255.0

label_binarizer = LabelBinarizer()
train_y = label_binarizer.fit_transform(train_y)

train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, train_size=0.8)
train_x = train_x.reshape((train_x.shape[0], 28 * 28 * 1))
valid_x = valid_x.reshape((valid_x.shape[0], 28 * 28 * 1))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [13]:
sequential_model = Sequential()
sequential_model.add(Dense(256, input_shape=(28 * 28 * 1,), activation="sigmoid"))
sequential_model.add(Dense(128, activation="sigmoid"))
sequential_model.add(Dense(10, activation="softmax"))

sequential_model.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)
sequential_model.fit(
    train_x,
    train_y,
    validation_data=(valid_x, valid_y),
    epochs=50,
    batch_size=256,
    verbose=0,
)
_, accuracy = sequential_model.evaluate(valid_x, valid_y, verbose=0)
print(f"Accuracy:{accuracy}")

Accuracy:0.9736904501914978
875/875 [==============================] - 1s 533us/step


In [36]:
r = sequential_model.predict(test_x)
print(r.shape)
# print(r)
size = 28000
answer = [np.argmax(row) for row in r]
print(answer[:100])

875/875 [==============================] - 1s 561us/step
(28000, 10)
[2, 0, 9, 9, 3, 7, 0, 3, 0, 3, 5, 7, 4, 0, 4, 3, 3, 1, 9, 0, 9, 1, 1, 5, 7, 4, 2, 7, 4, 7, 7, 5, 4, 2, 6, 2, 5, 5, 1, 6, 7, 7, 4, 9, 8, 7, 8, 2, 6, 7, 6, 8, 8, 3, 8, 2, 1, 2, 2, 5, 4, 1, 7, 0, 0, 0, 1, 9, 0, 1, 6, 5, 8, 8, 2, 8, 8, 9, 2, 3, 5, 9, 1, 0, 9, 2, 4, 3, 6, 7, 2, 0, 6, 6, 1, 4, 3, 9, 7, 4]


In [ ]:
answer = {"ImageId": list(range(1, size + 1)), "Label": answer}

answer = pd.DataFrame.from_dict(answer)
# print(answer)

answer.to_csv("answer.csv", index=False)